# DSCI525: Web and Cloud Computing

## Milestone I: Tackling big data on your laptop

*Authors (Group 3): Mitchie, Jianru, Aishwarya, Aditya*<br>
*Date: April 4th, 2021*

## Table of contents

- [1. Downloading the data](#1.-Downloading-the-data-)
- [2. Combine the data CSVs](#2.-Combine-the-data-CSVs)
- [3. Load the combined CSV to memory and perform a simple EDA](#3.-Load-the-combined-CSV-to-memory-and-perform-a-simple-EDA)
- [4. Perform a simple EDA in R](#4.-Perform-a-simple-EDA-in-R)
- [5. Final Comments](#5.-Final-Comments)


## Imports
<hr>

In [1]:
# Python imports
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
# Jupyter lab cell extensions
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\dengj\Miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


## 1. Downloading the data <a name="1"></a>

In [3]:
%pwd

'C:\\Users\\dengj\\ubc_mds_21\\ds_block6\\525_web_cloud_comp\\majacloud\\notebooks'

In [4]:
%cd C:\\Users\\dengj\\ubc_mds_21\\ds_block6\\525_web_cloud_comp\\majacloud\\notebooks

C:\Users\dengj\ubc_mds_21\ds_block6\525_web_cloud_comp\majacloud\notebooks


In [5]:
# Necessary metadata
article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [7]:
%%time

# get the file named data.zip
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 54.4 s


In [8]:
%%time

# extract the file
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 35.3 s


## 2. Combine the data CSVs

### 2.1 Combine the data CSVs by `Pandas`

In [9]:
# get an idea how individual file looks like 
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max", "rain (mm/day)"]
df1 = pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv", usecols=use_cols,dtype={'TailNum': 'str'})
df1['model'] = "ACCESS-CM2"
df1.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [11]:
out_combined_filepath = "figsharerainfall/combined_data.csv"

In [12]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together using pandas
files = glob.glob('figsharerainfall\*_daily_rainfall_NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'[^\\]+(?=\_d)', file)[0])
                for file in files)
              )

# df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
#                 .assign(model=re.findall(r'[^\\]+(?=\_d)', file)[0])
#                 for file in files)
#               )

print('The pandas data frame has been concatenated!')


df.to_csv("figsharerainfall/combined_data.csv")

print(f"The combined data is saved in {out_combined_filepath}")

peak memory: 491.90 MiB, increment: 0.12 MiB
The pandas data frame has been concatenated!
The combined data is saved in figsharerainfall/combined_data.csv
Wall time: 10min 43s


In [13]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.7G	figsharerainfall/combined_data.csv


In [14]:
%%time
df = pd.read_csv(out_combined_filepath)

Wall time: 1min 56s


In [15]:
print(df.shape)

(62467843, 7)


In [16]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


### 2.2 Combine the data CSV by `dask`

In [17]:
import dask.dataframe as dd
import pandas as pd

In [18]:
%%time
%%memit

# show time that dask take to merge

ddf = dd.read_csv("figsharerainfall/*_daily_rainfall_NSW.csv", usecols=use_cols, assume_missing=True)

out_ddf_filepath = "figsharerainfall/combined_data_dask.csv"
ddf.to_csv(out_ddf_filepath, single_file=True)

print(f"Yay dask combining to csv is done, and u can find the file here {out_ddf_filepath}")

Yay dask combining to csv is done, and u can find the file here figsharerainfall/combined_data_dask.csv
peak memory: 3164.26 MiB, increment: 1989.64 MiB
Wall time: 8min 36s


In [19]:
ddf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, time to rain (mm/day)
dtypes: object(1), float64(5)

### 2.3 Summary of observations on run-time and memory usages in both options (THIS IS A DRAFT TO HAVE EVERYONE'S INPUT AND MAY COMPILE LATER)
Compare run times and memory usages of these options on different machines within your team, and summarize your observations in your milestone notebook.

- Mitchie:

- Jianru: The runtime and memory usages can vary by the type of machines and operation systems, but relatively long time and fairly large memory are required for such big-sized data. `Dask` for loading and writing data is relatively faster than `Pandas`, due to the fact that `Dask` reads data in chunks and parallel. Overall, compiling such big data into one csv file and load it every single time is not suggested, and alternatively file type should be considered such as `feather`or `parquet`.

- Aishwarya:

- Aditya:

- Overall, TODO

## 3. Load the combined CSV to memory and perform a simple EDA

In [20]:
df.model

0           ACCESS-CM2
1           ACCESS-CM2
2           ACCESS-CM2
3           ACCESS-CM2
4           ACCESS-CM2
               ...    
62467838       TaiESM1
62467839       TaiESM1
62467840       TaiESM1
62467841       TaiESM1
62467842       TaiESM1
Name: model, Length: 62467843, dtype: object

### 3.1 Load only the interested columns

In [21]:
%%time
%%memit
#48s

data_model = pd.read_csv(out_combined_filepath, usecols=['model'])
print(data_model.model.value_counts())


MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
AWI-ESM-1-1-LR       966420
NESM3                966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
Name: model, dtype: int64
peak memory: 1054.27 MiB, increment: 954.12 MiB
Wall time: 59.4 s


### 3.2 Change `dtype` of the data

In [27]:
f64_mem = df.iloc[:, 1:6].memory_usage().sum()/1e6
f32_mem = df.iloc[:, 1:6].astype('float32', errors='ignore').memory_usage().sum() / 1e6

In [28]:
print(f"Memory usage with float64: {f64_mem: .2f} MB")
print(f"Memory usage with float32: {f32_mem: .2f} MB")

Memory usage with float64:  2498.71 MB
Memory usage with float32:  1249.36 MB


### 3.3 Load in chunks

In [24]:
%%time
%%memit
# 4min
# example of how to load data in small chunks. We can pass chunksize as a parameter while using read_csv.

model_counts = pd.Series(dtype=int)

for chunk in pd.read_csv('figsharerainfall/combined_data.csv', chunksize = 10_000_000):
    model_counts = model_counts.add(chunk['model'].value_counts(), fill_value = 0)
print(model_counts.astype(int))


ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
peak memory: 2827.38 MiB, increment: 2160.73 MiB
Wall time: 1min 32s


In [25]:
# Jianru py EDA 

### 3.4 Load the entire data using `dask`

In [26]:
%%time
%%memit
# 55s
ddf = dd.read_csv(out_combined_filepath)
print(ddf.model.value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 1997.25 MiB, increment: 1497.16 MiB
Wall time: 1min 1s


### 3.5 Discussion of the observations on those approaches

- Mitchie:

- Jianru: When only a certain columns are loaded, it takes shorter time to run for around 60s. 

- Aishwarya:

- Aditya:

- Overall, TODO

## 4. Perform a simple EDA in R

In [ ]:
# Aditya R EDA

### 4.1 (One approach to transfer df from py to R)

### 4.2 Discussion on why you choose this approach over the others

- Mitchie:

- Jianru: As the input data size is around 5.7GB, I would be extremely slow using `Pandas` exchange, so it's not considered. For `Parquet` file, it could have svaed more storage space, but this might not be the main concern here as the data size is manageable so far.

- Aishwarya:

- Aditya:

- Overall, TODO

## 5. Final Comments